In [1]:
import os
import uuid
import pickle

import pandas as pd

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [2]:
year = 2021
month = 3
taxi_type = 'green'

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
# input_file = f'https://s3.amazonaws.com/nyc-tlc/trip+data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

RUN_ID = os.getenv('RUN_ID', '820d3f851eb6426d9c1dfb2e1bf4d9a9')

In [3]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids


def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))

    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [ ]:
def load_model(run_id):
    logged_model = f's3://mlflow-models-maksat/1/{run_id}/artifacts/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id

    df_result.to_parquet(output_file, index=False)

In [5]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

In [6]:
!ls output/green

2021-01.parquet  2021-02.parquet  2021-03.parquet


In [7]:
pd.read_parquet(output_file)

,ride_id,lpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,predicted_duration,diff,model_version
0,a484cea2-fff7-4ad1-895b-f904afdda374,2021-03-01 00:05:42,83,129,8.350000,9.728988,-1.378988,820d3f851eb6426d9c1dfb2e1bf4d9a9
1,c47ea6a8-2e61-4b40-a195-7144d1d3f845,2021-03-01 00:21:03,243,235,5.233333,6.889894,-1.656561,820d3f851eb6426d9c1dfb2e1bf4d9a9
2,ca955e54-7259-4247-9e5f-462b7cfb081b,2021-03-01 00:02:06,75,242,20.333333,27.097545,-6.764211,820d3f851eb6426d9c1dfb2e1bf4d9a9
3,ae16e51b-38c8-4d67-9e31-51ba43fa34ff,2021-03-01 00:24:03,242,208,7.666667,14.386332,-6.719665,820d3f851eb6426d9c1dfb2e1bf4d9a9
4,7ba26c44-a832-4866-88b5-03c1dad65e60,2021-03-01 00:11:10,41,151,3.600000,5.557552,-1.957552,820d3f851eb6426d9c1dfb2e1bf4d9a9
...,...,...,...,...,...,...,...,...
80367,4ba25af3-e78e-4edb-b2ad-c20545c1e75a,2021-03-31 22:07:00,41,75,6.000000,9.028581,-3.028581,820d3f851eb6426d9c1dfb2e1bf4d9a9
80368,02958cff-1cb8-4f2e-832f-12ea9ca24f3b,2021-03-31 22:56:00,95,95,17.000000,1.786509,15.213491,820d3f851eb6426d9c1dfb2e1bf4d9a9
80369,bae9028b-64db-4d3d-abe1-4e34502eb607,2021-03-31 22:36:00,95,95,9.000000,4.850141,4.149859,820d3f851eb6426d9c1dfb2e1bf4d9a9
80370,abbb1718-525c-4872-8b90-acd3054cac45,2021-03-31 23:35:00,37,14,25.000000,27.339816,-2.339816,820d3f851eb6426d9c1dfb2e1bf4d9a9
